In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 21.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
import numpy as np
import time
from datetime import datetime

class SimpleASRModel(nn.Module):
    """
    A simplified ASR model using an LSTM.
    Note: When using raw waveforms, the input dimension is 1.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleASRModel, self).__init__()
        # LSTM layer expects input shape (batch, time, feature)
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        # Fully connected layer to map hidden state to output classes
        self.fc = nn.Linear(hidden_dim, output_dim)

        print(f"[MODEL] Initialized SimpleASRModel with input_dim={input_dim}, hidden_dim={hidden_dim}, output_dim={output_dim}")

    def forward(self, x):
        # x shape: (batch, time, feature)
        outputs, _ = self.lstm(x)
        # Use the last time-step for classification
        logits = self.fc(outputs[:, -1, :])
        return logits

def load_data(dataset_name, split):
    """
    Load a dataset from Hugging Face using the datasets library.

    Parameters:
      - dataset_name: the identifier of the dataset
          (e.g., 'united-we-care/United-Syn-Med' or 'mozilla-foundation/common_voice_11_0')
      - split: the dataset split to load (e.g., 'train')

    Returns:
      The loaded dataset.
    """
    print(f"[DATA] Loading dataset: {dataset_name}, split: {split}")
    start_time = time.time()

    try:
        dataset = load_dataset(dataset_name, split=split)
        load_time = time.time() - start_time
        print(f"[DATA] Successfully loaded dataset in {load_time:.2f} seconds")
        print(f"[DATA] Dataset size: {len(dataset)} examples")
        print(f"[DATA] First example keys: {list(dataset[0].keys())}")
        return dataset
    except Exception as e:
        print(f"[ERROR] Failed to load dataset: {e}")
        raise

def collate_fn(batch):
    """
    A collate function to pad variable-length audio sequences.

    This function supports datasets that provide:
      - Audio data under 'speech' or under 'audio' (with key 'array')
      - Transcriptions under 'transcript' or 'sentence'

    It pads the NumPy arrays using np.pad, then converts them to a tensor
    and unsqueezes the last dimension to add a feature dimension.
    """
    # Log batch size
    print(f"[COLLATE] Processing batch of size {len(batch)}")

    # Determine the audio field to use
    if 'speech' in batch[0]:
        print("[COLLATE] Using 'speech' field for audio data")
        speeches = [np.array(item['speech']) for item in batch]
    elif 'audio' in batch[0]:
        print("[COLLATE] Using 'audio.array' field for audio data")
        # Assume audio is stored as a dictionary with an 'array' key
        speeches = [np.array(item['audio']['array']) for item in batch]
    else:
        error_msg = f"[ERROR] No recognized audio field in dataset item. Available keys: {list(batch[0].keys())}"
        print(error_msg)
        raise ValueError(error_msg)

    # Determine the transcript field to use
    if 'transcript' in batch[0]:
        print("[COLLATE] Using 'transcript' field for text data")
        transcripts = [item['transcript'] for item in batch]
    elif 'sentence' in batch[0]:
        print("[COLLATE] Using 'sentence' field for text data")
        transcripts = [item['sentence'] for item in batch]
    else:
        error_msg = f"[ERROR] No recognized transcript field in dataset item. Available keys: {list(batch[0].keys())}"
        print(error_msg)
        raise ValueError(error_msg)

    # Find the maximum sequence length in the batch
    max_len = max(speech.shape[0] for speech in speeches)
    print(f"[COLLATE] Maximum sequence length in batch: {max_len}")

    # Print audio statistics
    min_len = min(speech.shape[0] for speech in speeches)
    avg_len = sum(speech.shape[0] for speech in speeches) / len(speeches)
    print(f"[COLLATE] Audio length statistics - Min: {min_len}, Max: {max_len}, Avg: {avg_len:.2f}")

    # Pad each audio sequence with zeros so all have the same length
    padded_speeches = [np.pad(speech, (0, max_len - speech.shape[0]), mode='constant') for speech in speeches]
    # Convert list to a single NumPy array before converting to tensor
    padded_speeches = np.array(padded_speeches)
    padded_speeches = torch.tensor(padded_speeches, dtype=torch.float32)
    # Add an extra dimension to represent the feature dimension (channel)
    padded_speeches = padded_speeches.unsqueeze(-1).contiguous()

    print(f"[COLLATE] Final tensor shape: {padded_speeches.shape}")
    print(f"[COLLATE] Sample transcript: '{transcripts[0][:50]}{'...' if len(transcripts[0]) > 50 else ''}'")

    return padded_speeches, transcripts

def create_dataloader(dataset, batch_size=16, shuffle=True):
    """
    Create a DataLoader for the given dataset.
    """
    print(f"[DATALOADER] Creating DataLoader with batch_size={batch_size}, shuffle={shuffle}")
    try:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
        print(f"[DATALOADER] Successfully created DataLoader with {len(dataloader)} batches")
        return dataloader
    except Exception as e:
        print(f"[ERROR] Failed to create DataLoader: {e}")
        raise

def train(model, dataloader, epochs=10, learning_rate=0.001, device='cpu'):
    """
    A simple training loop for the ASR model.

    For demonstration purposes, this loop uses dummy targets.
    In an actual ASR system, you would perform sequence-to-sequence training.

    Parameters:
      - model: the ASR model to train.
      - dataloader: the DataLoader for training data.
      - epochs: number of training epochs.
      - learning_rate: learning rate for the optimizer.
      - device: device to run training on (cpu or cuda).
    """
    print(f"\n[TRAIN] Starting training with {epochs} epochs, learning_rate={learning_rate}, device={device}")
    print(f"[TRAIN] Model will process {len(dataloader)} batches per epoch")

    # Check if CUDA is available if device is set to 'cuda'
    if device == 'cuda' and not torch.cuda.is_available():
        print("[WARNING] CUDA requested but not available. Falling back to CPU.")
        device = 'cpu'

    model.to(device)
    print(f"[TRAIN] Model moved to device: {device}")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f"[TRAIN] Using Adam optimizer with learning_rate={learning_rate}")

    model.train()
    print("[TRAIN] Model set to training mode")

    total_start_time = time.time()

    for epoch in range(epochs):
        epoch_start_time = time.time()
        print(f"\n[TRAIN] Starting Epoch {epoch+1}/{epochs} at {datetime.now().strftime('%H:%M:%S')}")

        epoch_loss = 0.0
        batch_count = 0

        for batch_idx, (inputs, transcripts) in enumerate(dataloader):
            batch_start_time = time.time()
            print(f"[TRAIN] Epoch {epoch+1}, Batch {batch_idx+1}/{len(dataloader)}")

            inputs = inputs.to(device)
            inputs = inputs.contiguous()
            print(f"[TRAIN] Input shape: {inputs.shape}")

            # Dummy target: using class 0 for all examples
            targets = torch.zeros(inputs.size(0), dtype=torch.long).to(device)
            print(f"[TRAIN] Created dummy targets with shape: {targets.shape}")

            # Forward pass
            print("[TRAIN] Starting forward pass")
            outputs = model(inputs)
            print(f"[TRAIN] Forward pass complete, output shape: {outputs.shape}")

            # Compute loss
            loss = criterion(outputs, targets)
            print(f"[TRAIN] Batch loss: {loss.item():.4f}")

            # Backward pass
            print("[TRAIN] Starting backward pass")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print("[TRAIN] Backward pass complete")

            epoch_loss += loss.item()
            batch_count += 1

            batch_time = time.time() - batch_start_time
            print(f"[TRAIN] Batch processed in {batch_time:.2f} seconds")

            # Print progress every 5 batches or for the last batch
            if (batch_idx + 1) % 5 == 0 or batch_idx + 1 == len(dataloader):
                avg_loss = epoch_loss / batch_count
                print(f"[TRAIN] Epoch {epoch+1}, Progress: {batch_idx+1}/{len(dataloader)} batches, Avg Loss: {avg_loss:.4f}")

        epoch_time = time.time() - epoch_start_time
        epoch_avg_loss = epoch_loss / len(dataloader)
        print(f"[TRAIN] Epoch {epoch+1}/{epochs} completed in {epoch_time:.2f} seconds")
        print(f"[TRAIN] Epoch {epoch+1} average loss: {epoch_avg_loss:.4f}")

    total_time = time.time() - total_start_time
    print(f"\n[TRAIN] Training complete! Total time: {total_time:.2f} seconds")
    print(f"[TRAIN] Average time per epoch: {total_time/epochs:.2f} seconds")

def evaluate(model, dataloader, device='cpu'):
    """
    Evaluate the model using a dummy accuracy metric.

    For a real ASR system, evaluation should involve decoding the output and computing metrics like WER.
    """
    print(f"\n[EVAL] Starting evaluation on {device}")

    # Check if CUDA is available if device is set to 'cuda'
    if device == 'cuda' and not torch.cuda.is_available():
        print("[WARNING] CUDA requested but not available. Falling back to CPU.")
        device = 'cpu'

    model.to(device)
    model.eval()
    print("[EVAL] Model set to evaluation mode")

    correct = 0
    total = 0
    eval_start_time = time.time()

    with torch.no_grad():
        for batch_idx, (inputs, transcripts) in enumerate(dataloader):
            print(f"[EVAL] Processing batch {batch_idx+1}/{len(dataloader)}")

            batch_start_time = time.time()
            inputs = inputs.to(device)
            print(f"[EVAL] Input shape: {inputs.shape}")

            # Dummy targets for demonstration
            targets = torch.zeros(inputs.size(0), dtype=torch.long).to(device)

            # Forward pass
            print("[EVAL] Running forward pass")
            outputs = model(inputs)
            print(f"[EVAL] Output shape: {outputs.shape}")

            # Get predictions
            _, predicted = torch.max(outputs, 1)
            print(f"[EVAL] Predictions shape: {predicted.shape}")

            # Update statistics
            batch_total = targets.size(0)
            batch_correct = (predicted == targets).sum().item()
            total += batch_total
            correct += batch_correct

            batch_time = time.time() - batch_start_time
            batch_accuracy = 100 * batch_correct / batch_total
            print(f"[EVAL] Batch {batch_idx+1} processed in {batch_time:.2f} seconds")
            print(f"[EVAL] Batch {batch_idx+1} accuracy: {batch_accuracy:.2f}% ({batch_correct}/{batch_total})")

    accuracy = 100 * correct / total if total > 0 else 0
    eval_time = time.time() - eval_start_time
    print(f"\n[EVAL] Evaluation completed in {eval_time:.2f} seconds")
    print(f"[EVAL] Overall accuracy: {accuracy:.2f}% ({correct}/{total})")


In [ ]:
import torch
# from asr_training import SimpleASRModel, load_data, create_dataloader, train, evaluate
from datasets import load_dataset
import time
from datetime import datetime

def load_common_voice():
    """
    Load and prepare the Common Voice dataset from Kaggle or local directory.
    This dataset provides diverse English accents.
    """
    print(f"[DATASET] Starting Common Voice dataset loading at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = time.time()

    # If downloaded from Kaggle
    # common_voice_path = "path/to/common_voice"

    # Alternative: Load directly from Hugging Face
    # common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "en")
    try:
        print("[DATASET] Attempting to load Common Voice dataset from Hugging Face")
        print("[DATASET] Parameters: mozilla-foundation/common_voice_11_0, language=en, num_proc=1, split=train")

        common_voice = load_dataset(
            "mozilla-foundation/common_voice_11_0",
            "en",
            num_proc=1,  # Reduce parallel processes
            trust_remote_code=True,  # Trust the remote code
            split="train",  # Use the training split
        )

        load_time = time.time() - start_time
        print(f"[DATASET] Successfully loaded Common Voice dataset in {load_time:.2f} seconds")
        print(f"[DATASET] Dataset size: {len(common_voice)} examples")

        # Print sample information
        if len(common_voice) > 0:
            print(f"[DATASET] First example keys: {list(common_voice[0].keys())}")
            # Print audio information for the first example
            if 'audio' in common_voice[0]:
                print(f"[DATASET] First example audio sampling rate: {common_voice[0]['audio']['sampling_rate']} Hz")
                print(f"[DATASET] First example audio array shape: {len(common_voice[0]['audio']['array'])}")
            # Print text information for the first example
            if 'sentence' in common_voice[0]:
                print(f"[DATASET] First example sentence: '{common_voice[0]['sentence'][:50]}{'...' if len(common_voice[0]['sentence']) > 50 else ''}'")

    except Exception as e:
        print(f"[ERROR] Failed to load dataset: {str(e)}")
        raise

    return common_voice

def main():
    print("\n[MAIN] Starting ASR model training and evaluation")
    print(f"[MAIN] Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"[MAIN] PyTorch version: {torch.__version__}")
    print(f"[MAIN] CUDA available: {torch.cuda.is_available()}")

    # Dataset identifiers
    common_voice_dataset_id = "mozilla-foundation/common_voice_11_0"

    # Disable cuDNN
    print("[MAIN] Disabling cuDNN")
    torch.backends.cudnn.enabled = False
    print(f"[MAIN] cuDNN enabled: {torch.backends.cudnn.enabled}")

    # Step 1: Load dataset
    print("\n[MAIN] Step 1: Loading Common Voice dataset")
    try:
        dataset_start_time = time.time()
        dataset = load_common_voice()
        dataset_time = time.time() - dataset_start_time
        print(f"[MAIN] Dataset loading completed in {dataset_time:.2f} seconds")
    except Exception as e:
        print(f"[MAIN] Failed to load dataset: {e}")
        return

    # Step 2: Create DataLoader
    print("\n[MAIN] Step 2: Creating DataLoader")
    try:
        batch_size = 8
        print(f"[MAIN] Creating DataLoader with batch_size={batch_size}")
        dataloader_start_time = time.time()
        dataloader = create_dataloader(dataset, batch_size=batch_size)
        dataloader_time = time.time() - dataloader_start_time
        print(f"[MAIN] DataLoader creation completed in {dataloader_time:.2f} seconds")
        print(f"[MAIN] Number of batches: {len(dataloader)}")
    except Exception as e:
        print(f"[MAIN] Failed to create DataLoader: {e}")
        return

    # Step 3: Define model hyperparameters
    print("\n[MAIN] Step 3: Defining model hyperparameters")
    input_dim = 1      # Example: number of MFCC features (placeholder)
    hidden_dim = 60   # Hidden dimension for LSTM
    output_dim = 10    # Example: number of output classes (placeholder)

    print(f"[MAIN] Model hyperparameters:")
    print(f"[MAIN] - input_dim: {input_dim}")
    print(f"[MAIN] - hidden_dim: {hidden_dim}")
    print(f"[MAIN] - output_dim: {output_dim}")

    # Step 4: Initialize the model
    print("\n[MAIN] Step 4: Initializing the model")
    try:
        model_start_time = time.time()
        model = SimpleASRModel(input_dim, hidden_dim, output_dim)
        model_time = time.time() - model_start_time
        print(f"[MAIN] Model initialization completed in {model_time:.2f} seconds")
        print(f"[MAIN] Model architecture:\n{model}")

        # Count model parameters
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"[MAIN] Total parameters: {total_params:,}")
        print(f"[MAIN] Trainable parameters: {trainable_params:,}")
    except Exception as e:
        print(f"[MAIN] Failed to initialize model: {e}")
        return

    # Step 5: Set the device
    print("\n[MAIN] Step 5: Setting up device")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[MAIN] Using device: {device}")

    if device.type == 'cuda':
        print(f"[MAIN] CUDA Device: {torch.cuda.get_device_name(0)}")
        print(f"[MAIN] CUDA Memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
        print(f"[MAIN] CUDA Memory cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")

    # Step 6: Train the model
    print("\n[MAIN] Step 6: Training the model")
    try:
        epochs = 5
        learning_rate = 0.001
        print(f"[MAIN] Training parameters:")
        print(f"[MAIN] - epochs: {epochs}")
        print(f"[MAIN] - learning_rate: {learning_rate}")
        print(f"[MAIN] - device: {device}")

        train_start_time = time.time()
        train(model, dataloader, epochs=epochs, learning_rate=learning_rate, device=device)
        train_time = time.time() - train_start_time
        print(f"[MAIN] Training completed in {train_time:.2f} seconds")
        print(f"[MAIN] Average time per epoch: {train_time/epochs:.2f} seconds")

        if device.type == 'cuda':
            print(f"[MAIN] CUDA Memory after training: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    except Exception as e:
        print(f"[MAIN] Error during training: {e}")
        return

    # Step 7: Evaluate the model
    print("\n[MAIN] Step 7: Evaluating the model")
    try:
        eval_start_time = time.time()
        evaluate(model, dataloader, device=device)
        eval_time = time.time() - eval_start_time
        print(f"[MAIN] Evaluation completed in {eval_time:.2f} seconds")
    except Exception as e:
        print(f"[MAIN] Error during evaluation: {e}")
        return

    # Step 8: Summarize execution
    total_time = time.time() - dataset_start_time
    print("\n[MAIN] Execution summary:")
    print(f"[MAIN] Total execution time: {total_time:.2f} seconds")
    print(f"[MAIN] - Dataset loading: {dataset_time:.2f} seconds ({dataset_time/total_time*100:.1f}%)")
    print(f"[MAIN] - DataLoader creation: {dataloader_time:.2f} seconds ({dataloader_time/total_time*100:.1f}%)")
    print(f"[MAIN] - Model initialization: {model_time:.2f} seconds ({model_time/total_time*100:.1f}%)")
    print(f"[MAIN] - Training: {train_time:.2f} seconds ({train_time/total_time*100:.1f}%)")
    print(f"[MAIN] - Evaluation: {eval_time:.2f} seconds ({eval_time/total_time*100:.1f}%)")
    print(f"[MAIN] Process completed successfully at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    return model

if __name__ == "__main__":
    try:
        model = main()
        print("[MAIN] Model training and evaluation completed successfully")
        # Save the model as pickle
        import pickle
        with open("asr_model.pkl", "wb") as f:
            pickle.dump(model, f)
    except Exception as e:
        print(f"[FATAL] Unhandled exception in main: {e}")
        import traceback
        print(traceback.format_exc())


[MAIN] Starting ASR model training and evaluation
[MAIN] Current time: 2025-04-07 20:15:07
[MAIN] PyTorch version: 2.6.0+cu124
[MAIN] CUDA available: True
[MAIN] Disabling cuDNN
[MAIN] cuDNN enabled: False

[MAIN] Step 1: Loading Common Voice dataset
[DATASET] Starting Common Voice dataset loading at 2025-04-07 20:15:07
[DATASET] Attempting to load Common Voice dataset from Hugging Face
[DATASET] Parameters: mozilla-foundation/common_voice_11_0, language=en, num_proc=1, split=train


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
[MODEL] Initialized SimpleASRModel with input_dim=1, hidden_dim=60, output_dim=10
[MAIN] Model initialization completed in 0.00 seconds
[MAIN] Model architecture:
SimpleASRModel(
  (lstm): LSTM(1, 60, batch_first=True)
  (fc): Linear(in_features=60, out_features=10, bias=True)
)
[MAIN] Total parameters: 15,730
[MAIN] Trainable parameters: 15,730

[MAIN] Step 5: Setting up device
[MAIN] Using device: cuda
[MAIN] CUDA Device: NVIDIA A100-SXM4-40GB
[MAIN] CUDA Memory allocated: 0.00 MB
[MAIN] CUDA Memory cached: 0.00 MB

[MAIN] Step 6: Training the model
[MAIN] Training parameters:
[MAIN] - epochs: 5
[MAIN] - learning_rate: 0.001
[MAIN] - device: cuda

[TRAIN] Starting training with 5 epochs, learning_rate=0.001, device=cuda
[TRAIN] Model will process 118592 batches per epoch
[TRAIN] Model moved to device: cuda
[TRAIN] Using Adam optimizer with learning_rate=0.001
[TRAIN] Model set to training mode

[TRAIN] Starting Epoch 1/5 at 20:15:12
